In [563]:
# !pip install soynlp
# !pip install gensim

In [1]:
import pandas as pd
import numpy as np
import re
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus


tokenizer = RegexTokenizer()
tokenizer

In [2]:
# 곡 정보 파일 불러오기
df = pd.read_csv('data/song_tidy03.csv')

In [3]:
df.shape

(9582, 12)

In [4]:
# 가수 정보 파일 불러오기
df2 = pd.read_csv('data/artist_info_combined_ver04.csv', encoding='euc-kr')

In [5]:
# 곡 정보와 가수 정보를 머지해준다.
song = pd.merge(df, df2, on='artist')

In [6]:
# 가사 분석에 용이하도록 <br>을 공백으로 바꿔준다.
song['lyrics'] = song['lyrics'].str.replace(r'<br>', ' ')

In [214]:
whole_song = song['lyrics']
f = song[song['gender'] == '여']
whole_female = f['lyrics'].reset_index(drop=True)
m = song[song['gender'] == '남']
whole_male = m['lyrics'].reset_index(drop=True)

In [79]:
# 여자 전체곡수 
female_len = whole_female.shape[0]
# 남자 전체곡수 
male_len = whole_male.shape[0]

In [80]:
# 타이틀 곡은 몇곡일까
len(song[song.is_title == '타이틀 곡'])

2379

## 해당 단어를 포함한 곡의 수 

In [86]:
female_k = whole_female[whole_female['lyrics'].str.contains("본능", na=False)]
female_k = int(female_k.shape[0])

In [87]:
male_k = whole_male[whole_male['lyrics'].str.contains("본능", na=False)]
male_k = int(male_k.shape[0])

In [90]:
# 여자그룹의 가수 곡 중에서 해당 단어의 등장 비율 보기 
(female_k/female_len)*100  # 0.4퍼 

0.4194128220491312

In [91]:
# 남자그룹의 가수 곡 중에서 해당 단어의 등장 비율 보기 
(male_k/male_len)*100  # 1퍼 

0.9712046345203612

## 단어 전처리 및 토크나이징

In [16]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [17]:
def tokenize(df):
    sentences = df['lyrics'].apply(preprocessing)
    tokens = sentences.apply(tokenizer.tokenize)
    tokens = tokens.tolist()
    tokens_all = sum(tokens, [])
    return tokens_all

In [18]:
# 토크나이징 적용

song_t = tokenize(song)  # 토크나이징 된 것이 해당 song_t로 들어간다.
female_t = tokenize(female)
male_t = tokenize(male)

# 하다가 꼬였는가? copy 한거 쓰자!!!!!

In [19]:
# copy()

song_t = song_t.copy()
female_t = female_t.copy()
male_t = male_t.copy()

In [20]:
len(female_t)  # 여그룹 총 토크나이징된 단어 769,211  약 77만 

769211

In [21]:
len(male_t)  # 남그룹 총 토크나이징된 단어  1,409,422  약 141만

1409422

In [698]:
# male_t  # 하나로 합쳐져있다.

## 앞/뒤에 오는 단어를 보는 함수

In [22]:
# 중복되는 구절이 있어서 만든 유니크한 값만 보는 함수

def unique(word):
    result = []
    for i in word:
        if i not in result: #inside for
            result.append(i) #inside for inside if
    return result
    print(result)

In [851]:
# # 앞 5글자, 뒤 5글자 한번에 보는 함수 
    
# def n_f_word(tokenized_list, search_word):
#     indices2 = [i for i, x in enumerate(tokenized_list) if x == search_word]
#     i = 0
#     result = []
#     for i in range(len(indices2)):
#         next_word_2 = tokenized_list[indices2[i] - 5 :indices2[i] + 6]  #
#         result.append(next_word_2[1:])
#         i+=1
#     return result


In [830]:
# # search_word 뒤에 무슨 단어가 있는지 보는 함수

# def next_word(tokenized_list, search_word):
#     indices2 = [i for i, x in enumerate(tokenized_list) if x == search_word]
#     i = 0
#     result = []
#     for i in range(len(indices2)):
#         next_word_2 = tokenized_list[indices2[i]:indices2[i] + 11]  # 뒤에 10글자까지만 보기 
#         result.append(next_word_2[1:])
#         i+=1
#     return result


In [831]:
# # search_word 앞에 무슨 단어가 있는지 보는 함수

# def before_word(tokenized_list, search_word):
#     indices2 = [i for i, x in enumerate(tokenized_list) if x == search_word]
#     i = 0
#     result = []
#     for i in range(len(indices2)):
#         next_word_2 = tokenized_list[indices2[i] - 10:indices2[i] ]  # 앞에 10글자까지만 보기 
#         result.append(next_word_2)
#         i+=1
#     return result
#     print(result)

In [833]:
# # 두 음절 이상의 단어중에서 빈도수 상위 30 번째까지 출력 

# topn = 30 

# def more_word(word):     
#     more_words = []
#     for v1 in word:
#         for v2 in v1:
#              if len(v2) > 1:
#                 more_words.append(v2)
#     result = Counter(more_words).most_common(topn)
#     return result

In [26]:
# # search_word 앞,뒤에 무슨 단어가 있는지 보는 함수

import re

def before_next_word(tokens, pattern):
    indices = [i for i, x in enumerate(tokens) if re.match(pattern, x) is not None]
    result = [tokens[i - 5:i + 6] for i in indices]
    return result


# 원하는 키워드가 들어간 문장 '한 노래당 하나씩만' 보기 

In [147]:
# EX) '잘생' 이라는 글자가 등장하면 곡 당 제일 먼저 걸리는 한 구절만 가져오게 했습니다.

In [176]:
len(lyrics)

9566

In [215]:
len(whole_male)

5869

In [216]:
lyrics = whole_female  # whole_song(전체곡 중에서 보기), whole_female(여성그룹 중에서 보기), whole_male(남성그룹 중에서 보기)
result = []

import re

for i in range(len(lyrics)):
    texts = lyrics[i]
    f = re.search('(.{0,40}잘생.{0,40})', texts)
    if f is not None:
        result.append('노래번호 (' + str(i) + ') ' + f[0])

In [217]:
len(result) # 등장한 곡 수 

13

In [201]:
result  # EX) '잘생' 이라는 글자가 등장하면 곡 당 제일 먼저 걸리는 한 구절만 가져오게 했습니다.

['노래번호 (315) 니 파써니 피바가지 썼니 애들이 놀린다고 기죽지마 넌 이쁘고 착하고 또 잘생기고 멋있고 널 사랑해주는 내가 있잖아 힘내라 써니 장하다 써니 내 사랑',
 '노래번호 (357)  못생겨서 주눅이 드셨나  길고 짧은 건 대봐야 아는 법이지  키 크고 잘생긴 친구들 방심할때에  너에게 돌아가는 기회가 가는거야  도대체가 꼬일대',
 '노래번호 (401) t no playboy NO NO NO No playboy 어쩐지 너무 잘생겼어 말이 너무 달콤했어 쳐다보는 눈빛이 보통이 아니었어 너무나 자연스럽',
 '노래번호 (491) 런 바보였었지 아니야 아무도 나에게 사랑을 말해준 그런 사람 없었던거야 잘생긴 외모와 든든한 학벌과 집안 그런것들을 다 사랑해야할 그런 여자들과 세',
 '노래번호 (580) 좀 봐 또 빨개지네 요즘 따라 니가 왜 남자로 보일까 오늘따라 유독 왜 잘생겨 보일까 니가 너무 좋아 자꾸 안달이나 이게 사랑인가 말 안 해도 느낌',
 '노래번호 (877) 지 않나요 너무 멋있습니다 너무 멋있 아우 아우 어 누구 멤버지 얼굴도 잘생기고 춤도 잘 추고 노래도 잘하고 노래도 잘하고 이거 누구 멤버야 삼박자',
 '노래번호 (1512)  날 믿지 못하니 나는 죽을 때까지 널 사랑해 섹시한 당신은 나의 남자 잘생긴 당신은 나의 남자 이 세상 마지막이 온대도 영원히 이 환상에 젖을래 ',
 '노래번호 (1526) 그 아저씨군 삐뚤빼뚤 다같이 Stop 왼손은 허리위로 오른손은 머리위로 잘생긴 부처님 다같이 부처핸섬 오 아재 소금이 죽었소 맞춰봐 소금이 죽으면 ',
 '노래번호 (1843)  모두 털어줄게 거덜나게 우린 멋져부러 허벌나게 내 이름 진 끊임 없이 잘생김 근데 내 별명 찐따 But 최근 들어 따가 빠졌지 Yeah 이제 내 ',
 '노래번호 (2556) 정말 마지막으로 한 마디만 할게 그때 그 사람 아니 그때 그 놈이 맞지 잘생기긴 했던데 그리 듬직해 보이진 않던데 말야 네가 무슨 잘못이니 너무 착',
 '노래번호 (3439) ke

In [157]:
lyrics[315]  # 전체 가사로 확인해보기 

'써니 써니야 뭐가 불만이니 니 이름이 얼마나 좋은데 왜 고기써니 파써니 피바가지 썼니 애들이 놀린다고 기죽지마 넌 이쁘고 착하고 또 잘생기고 멋있고 널 사랑해주는 내가 있잖아 힘내라 써니 장하다 써니 내 사랑 써니 만만세 써니 니 이름이 얼마나 좋은 이름이니 모써니씨는 최연소 시의원도 됐잖아 차표한장 던져놓고 왜 떠났냐고 혹 기억나니 대관이 아찌 마구 울부짖었잖아 이렇게 가방 끌어안고 있는 날 보고 뭘 생각하니 느껴 내가 불쌍하지 않아 나 울지않게 해줘 제발 부탁이야 써니 나의 사랑 나의 신부 써니야  써니 이름 바꿔달라고 떼쓰지마 써니는 뭐 그냥 써니지 뭐 뭐 먹니가 되겠니 죽기가 되겠니 그러니까 제발 엄마 아빠 괴롭히지마 너는 내 곁에서 언제나 착하고 예쁜 그 모습 더도말고 그대로만 내게 있어줘 잊지마 너에겐 내가 있는걸 언제나 네 곁엔 내가 있어줄게 써니 이제 내가 니 이름 부를날도 얼마 남지도 않았는데 자기, 여보, 당신, 어이! 앞으로 나도 이렇게 부를테고 맘마, 엄마, 아줌마, 할머니 이제 니 이름도 거의 사라질텐데 니 얼굴, 마음 그리고 이쁜 너의 이름 더욱더 아껴줘야 해요 써니!!!'

In [140]:
len(before_next_word(song_t, r'(잘생)'))  # 한노래에 있는 구절이 여러개 중복해서 뽑힐 수 있다. 

40

In [145]:
unique(before_next_word(song_t, r'(잘생)'))[:5]

[['기죽지마', '넌', '이쁘고', '착하고', '또', '잘생기고', '멋있고', '널', '사랑해주는', '내가', '있잖아'],
 ['대봐야', '아는', '법이지', '키', '크고', '잘생긴', '친구들', '방심할때에', '너에게', '돌아가는', '기회가'],
 ['NO',
  'No',
  'playboy',
  '어쩐지',
  '너무',
  '잘생겼어',
  '말이',
  '너무',
  '달콤했어',
  '쳐다보는',
  '눈빛이'],
 ['사랑을',
  '말해준',
  '그런',
  '사람',
  '없었던거야',
  '잘생긴',
  '외모와',
  '든든한',
  '학벌과',
  '집안',
  '그런것들을'],
 ['남자로', '보일까', '오늘따라', '유독', '왜', '잘생겨', '보일까', '니가', '너무', '좋아', '자꾸']]

#  both_words_ratio함수로는 
# (1) 두 그룹 모두에게서 등장한 단어들의 비율을 보거나 
# (2) 한 그룹에서만 등장한 단어들의 비율을 볼 수 있습니다. 
# 원하는 부분을 제외하고 주석 처리 해주세요.

In [834]:
# both_words_ratio(female_back_words, male_back_words, female_back_len, male_back_len)

def both_words_ratio(f_, m_, f_count, m_count):
       
    f = pd.DataFrame(f_)
    m = pd.DataFrame(m_)
    f.columns = ['words', 'count_f']
    m.columns = ['words', 'count_m']

# (1) 두 그룹 모두에게서 등장한 단어들의 비율을 보는 함수

    merged_in = pd.merge(f, m, how='inner', on=['words'])    
    f['count_f'] = (merged_in['count_f']/f_count)*100
    m['count_m'] = (merged_in['count_m']/m_count)*100
    
    f = f.dropna()
    m = m.dropna()
    result = pd.merge(f, m, how='inner', on=['words'])  
    
# # (2) 한 그룹에서만 등장한 단어들의 비율을 보는 함수    

#     merged_out = pd.merge(f, m, how='outer', on=['words'])    
#     f['count_f'] = (merged_in['count_f']/f_count)*100
#     m['count_m'] = (merged_in['count_m']/m_count)*100
#     result = pd.merge(f, m, how='outer', on=['words']) 

    return result

In [890]:
def excute_both_both(word):

    female_both_words = more_word(unique(n_f_word(female_t, word)))  
    male_both_words = more_word(unique(n_f_word(male_t, word)))
    
    female_both_len = len(n_f_word(female_t, word))
    male_both_len = len(n_f_word(male_t, word))
    
    return female_both_words, male_both_words, female_both_len, male_both_len
#     return (both_words_ratio(female_next_words, male_next_words, female_next_len, male_next_len))

In [835]:
def excute_before_both(word):


    female_before_words = more_word(unique(before_word(female_t, word)))  
    male_before_words = more_word(unique(before_word(male_t, word)))
    
    female_before_len = len(before_word(female_t, word))
    male_before_len = len(before_word(male_t, word))
    
    return (both_words_ratio(female_before_words, male_before_words, female_before_len, male_before_len))
    

In [836]:
def excute_next_both(word):


    female_next_words = more_word(unique(next_word(female_t, word)))  
    male_next_words = more_word(unique(next_word(male_t, word)))
    
    female_next_len = len(next_word(female_t, word))
    male_next_len = len(next_word(male_t, word))
    
    return (both_words_ratio(female_next_words, male_next_words, female_next_len, male_next_len))

In [ ]:
엉덩이,허리,몸매,다리

In [926]:
result0 = excute_before_both('엉덩이')  # 앞, 뒤 5단어씩 보기 
result0.to_excel('nlp_result/엉덩이.xlsx')

In [ ]:
f = pd.DataFrame(f_)
m = pd.DataFrame(m_)
f.columns = ['words', 'count_f']
m.columns = ['words', 'count_m']
    
    
merged_out = pd.merge(f, m, how='outer', on=['words'])    
f['count_f'] = (merged_in['count_f']/f_count)*100
m['count_m'] = (merged_in['count_m']/m_count)*100
result = pd.merge(f, m, how='outer', on=['words']) 

# 밑에는 코드연습해본것! 

In [628]:
찾을단어 = ['누나', '오빠']

In [629]:
dic_both_before = dict()  # 두 그룹에서 모두 등장한 단어
dic_parts_before = dict()  # 하나의 그룹에서만 등장한 단어 

for word in 찾을단어:
    female_before_words = more_word(unique(before_word(female_t, word)))  
    male_before_words = more_word(unique(before_word(male_t, word)))
    
    female_before_len = len(before_word(female_t, word))
    male_before_len = len(before_word(male_t, word))
    
    dic_both_before[word + "_앞_결과"] = both_words_ratio(female_before_words, male_before_words, female_before_len, male_before_len)
    dic_parts_before[word + "_앞_결과"] = parts_words_ratio(female_before_words, male_before_words, female_before_len, male_before_len)
    
    
    

In [560]:
dic_both_next = dict()  # 두 그룹에서 모두 등장한 단어
dic_parts_next = dict()  # 하나의 그룹에서만 등장한 단어 

for word in 찾을단어:
    female_back_words = more_word(unique(next_word(female_t, word)))  
    male_back_words = more_word(unique(next_word(male_t, word)))
    
    female_back_len = len(next_word(female_t, word))
    male_back_len = len(next_word(male_t, word))
    
    dic_both_next[word + "_뒤_결과"] = both_words_ratio(female_back_words, male_back_words, female_back_len, male_back_len)
    dic_parts_next[word + "_뒤_결과"] = parts_words_ratio(female_back_words, male_back_words, female_back_len, male_back_len)

In [554]:
dic_both_before['누나_앞_결과']

,words,count_f,count_m
0,우리,250.0,NaN


In [555]:
dic_both_before['오빠_앞_결과']

,words,count_f,count_m
0,오빠,80.45977,38.834951
1,나쁜,NaN,NaN


In [561]:
dic_parts_next.loc[dic_parts_next['누나_뒤_결과'].isnull()]

AttributeError: 'dict' object has no attribute 'loc'

# inner join했는데 왜 NaN이 나오지??  => 해결함

In [256]:
merged_out.loc[merged_out['count_f'].isnull()]  # 남자그룹에만 등장한 단어 

,words,count_f,count_m
30,벗어나고,NaN,6.802721
31,돌아가고,NaN,6.802721
32,안아주고,NaN,6.377551
33,지키고,NaN,5.952381
34,잊고,NaN,5.952381
35,네게,NaN,5.102041
36,머물고,NaN,5.102041


In [257]:
merged_out.loc[merged_out['count_m'].isnull()]  # 여자그룹에만 등장한 단어 

,words,count_f,count_m
17,너에게,8.124077,NaN
20,전하고,7.385524,NaN
21,영원히,6.646972,NaN
23,안기고,6.646972,NaN
24,이젠,6.646972,NaN
26,보여주고,5.908419,NaN
27,함께,5.908419,NaN


# soynlp_명사추출기ver2 사용해보기 

In [28]:
from soynlp.noun import LRNounExtractor
noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(sentences_all) # list of str like

used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
All 2398 r features was loaded
scanning completed
(L,R) has (50013, 23692) tokens
building lr-graph completed

In [29]:
nouns

{'영상': NounScore(frequency=8, score=0.505755, known_r_ratio=0.5),
 '루즈': NounScore(frequency=23, score=0.9989745333333333, known_r_ratio=1.0),
 '인간': NounScore(frequency=60, score=0.6711133939393938, known_r_ratio=0.7021276595744681),
 '울지': NounScore(frequency=427, score=0.9915242, known_r_ratio=0.09803921568627451),
 '의심': NounScore(frequency=169, score=0.7953841090909091, known_r_ratio=0.7534246575342466),
 '진행': NounScore(frequency=10, score=0.504724, known_r_ratio=0.25),
 '뜸들': NounScore(frequency=8, score=0.9995985000000001, known_r_ratio=0.2857142857142857),
 '깃발': NounScore(frequency=17, score=0.5694347499999999, known_r_ratio=1.0),
 '한장': NounScore(frequency=13, score=0.7171820000000001, known_r_ratio=0.875),
 '볼만': NounScore(frequency=16, score=0.9970192727272728, known_r_ratio=0.9166666666666666),
 '내면': NounScore(frequency=27, score=0.999769, known_r_ratio=0.5),
 '내품': NounScore(frequency=60, score=0.9655438723404256, known_r_ratio=0.8103448275862069),
 '거역': NounScore(freq

In [30]:
first_key = list(nouns.keys())[0]

In [31]:
first_key

'영상'

In [32]:
# 딕셔너리는 따로 순서가 없어서 인덱스 번호가 안먹는다. 따라서 따로 함수를 만들어서 단어만 추출함

def get_first_key(dictionary):
    for key in dictionary.keys():
        return key

keys = []  

for k in range(len(nouns)):
    key_name = list(nouns.keys())[k]
    keys.append(key_name)

In [35]:
len(keys)  # 원래 전체(명사,형용사...)토큰의 개수는 2,244,770 (약 224만개)

4254

In [37]:
df_keys = pd.DataFrame(keys)

In [44]:
df_keys[0].value_counts()  

어이        1
일루        1
혼잣말       1
삶속        1
편지        1
대구        1
챔피언       1
Cut       1
아줌마       1
사탕        1
허우적대      1
끝이나       1
아무일       1
변화구       1
욕심들       1
설득        1
그렇게       1
대접        1
주목        1
도착        1
jam       1
매너        1
현명        1
매력적       1
패배자       1
행복함       1
잠잘때       1
어둠        1
아쉬움       1
내모습       1
         ..
노을        1
KO        1
에에에에에에    1
비결        1
고정관념      1
니맘        1
허무        1
따뜻        1
경계        1
달콤함       1
신혼        1
땡땡        1
즐거움       1
여행        1
컴컴        1
생각들       1
낭비        1
웃을수       1
유리잔       1
눈가        1
환희        1
가졌다       1
다섯        1
큰사        1
우아        1
기침        1
한발짝       1
반대        1
뿌듯        1
고통        1
Name: 0, Length: 4254, dtype: int64